In [1]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings
from bs4 import BeautifulSoup
from tqdm import tqdm
import multiprocessing as mp
# from DataProcessing import 

warnings.filterwarnings('ignore')

In [2]:
state_codes = pd.read_excel('/home/vlad/csv/Us-zipcodes/statecode.xlsx',index_col=0)

In [8]:
#Unemployment

path_to_unemp = '/home/vlad/csv/US-Unemployement/us-county-unemployement20102018.xlsx'

unempdf = pd.concat([pd.read_excel(path_to_unemp,sheet_name=i,index_col=0) for i in range(9)],axis=1)

col_to_filt = [col for col in unempdf.columns if 'Pct' in col]

unempdf= unempdf[col_to_filt]
unempdf.columns = ['Unemp-'+col for col in unempdf.columns]

unempdf.index = [', '.join(ind.replace('County','').replace(' ','').strip().split(',')) 
                 for ind in unempdf.index]
unempdf.sort_index()

,Unemp-Pct-2014,Unemp-Pct-2015,Unemp-Pct-2016,Unemp-Pct-2017,Unemp-Pct-2018,Unemp-Pct-2013,Unemp-Pct-2012,Unemp-Pct-2011,Unemp-Pct-2010
"Abbeville, SC",7.7,6.9,5.5,4.6,4.0,9.5,10.6,12.5,13.6
"AcadiaParish, LA",5.8,6.8,7.4,5.9,5.4,6.0,6.0,6.7,7.3
"Accomack, VA",6.7,5.4,4.9,4.5,3.8,7.2,7.9,8.4,7.9
"Ada, ID",4.1,3.7,3.3,2.8,2.5,5.2,6.2,7.3,8.3
"Adair, IA",4.0,3.2,2.9,2.8,2.4,4.2,4.4,4.9,5.5
...,...,...,...,...,...,...,...,...,...
"Yuma, AZ",22.9,21.7,18.9,17.0,17.0,25.5,23.9,24.2,25.1
"Yuma, CO",3.3,2.6,2.1,1.5,2.0,4.8,5.1,5.3,5.6
"Zapata, TX",5.2,7.6,10.8,7.4,5.6,6.2,6.3,8.1,10.4
"Zavala, TX",12.1,11.2,13.9,11.2,9.5,14.5,13.5,14.1,13.9


In [4]:
#CPI Data
path_to_cpi = '/home/vlad/csv/US-CPI/SeriesReport-20191118124049_549fff.xlsx'
cpi_df = pd.read_excel(path_to_cpi,skiprows=11,index_col=0)['Annual'][:-2]; cpi_df

Year
1990    130.700
1991    136.200
1992    140.300
1993    144.500
1994    148.200
1995    152.400
1996    156.900
1997    160.500
1998    163.000
1999    166.600
2000    172.200
2001    177.100
2002    179.900
2003    184.000
2004    188.900
2005    195.300
2006    201.600
2007    207.342
2008    215.303
2009    214.537
2010    218.056
2011    224.939
2012    229.594
2013    232.957
2014    236.736
2015    237.017
2016    240.007
2017    245.120
Name: Annual, dtype: float64

In [5]:
#Income

path_to_income = '/home/vlad/csv/US-Personal-Income/CAINC1__ALL_STATES_1969_2017.xlsx'

raw_df = pd.read_excel(path_to_income
#                         ,encoding = "utf-8"
                       # ,error_bad_lines='ignore'
                       )

income_df = (raw_df.loc[(raw_df['Description']=='Per capita personal income (dollars) 2/') &
           (raw_df['GeoName'].str.contains(','))
          ].drop(columns=['GeoFIPS','Region','TableName','LineCode','IndustryClassification','Description','Unit'])
            .set_index('GeoName')
             .replace({'(NA)':np.nan})
             .astype(float)
             [cpi_df.index]
             
            )


income_df

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
GeoName,,,,,,,,,,,,,,,,,,,,,
"Autauga, AL",15482.0,16417.0,17063.0,17700.0,18683.0,19350.0,20155.0,21076.0,22073.0,22948.0,...,32876.0,32603.0,33415.0,34325.0,35040.0,35464.0,36677.0,38591.0,39509.0,40484.0
"Baldwin, AL",15831.0,16954.0,18107.0,19106.0,20129.0,21153.0,22342.0,23434.0,24643.0,25373.0,...,35751.0,34698.0,36282.0,37804.0,38166.0,38212.0,39561.0,41412.0,43004.0,44079.0
"Barbour, AL",13490.0,14180.0,14943.0,15349.0,15930.0,15912.0,16109.0,16670.0,17651.0,19015.0,...,25678.0,26088.0,27834.0,28106.0,28145.0,30040.0,29941.0,31509.0,31750.0,33453.0
"Bibb, AL",12672.0,12896.0,13494.0,13936.0,14686.0,15352.0,15892.0,16375.0,16154.0,16475.0,...,24983.0,23789.0,25091.0,25973.0,27028.0,27420.0,28249.0,29048.0,28989.0,30022.0
"Blount, AL",13797.0,14269.0,14761.0,15205.0,15923.0,16537.0,17663.0,18109.0,19265.0,19721.0,...,27661.0,27381.0,27754.0,28368.0,29600.0,30188.0,31359.0,31987.0,32261.0,33707.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Sweetwater, WY",18111.0,18439.0,19487.0,20244.0,21577.0,22176.0,23424.0,24952.0,26366.0,27538.0,...,47177.0,40995.0,43231.0,46916.0,47511.0,47736.0,48978.0,49256.0,47555.0,49870.0
"Teton, WY",35525.0,35656.0,38354.0,42881.0,43988.0,48664.0,51107.0,54243.0,62147.0,68794.0,...,152558.0,125633.0,145017.0,151738.0,194277.0,178164.0,200007.0,204913.0,224366.0,233860.0
"Uinta, WY",15083.0,16215.0,16638.0,17522.0,17758.0,18800.0,19298.0,20631.0,21484.0,22504.0,...,40931.0,38479.0,38607.0,39840.0,40041.0,39091.0,39191.0,38931.0,37866.0,38797.0


In [17]:
#CPI adjusted income 
base_year = 2010
income_df_adj = income_df.copy()
for year_ in income_df.columns:
    
    income_df_adj[year_] = (cpi_df[base_year] / cpi_df[year_]) * income_df[year_] 
    #income_df_adj[year_] = income_df[year_] / (1+((cpi_df[year_] - cpi_df[base_year]) / cpi_df[base_year]) )
    
income_df_adj.columns = ['Income_adj-'+str(col) for col in income_df_adj.columns]
income_df_adj     

,Income_adj-1990,Income_adj-1991,Income_adj-1992,Income_adj-1993,Income_adj-1994,Income_adj-1995,Income_adj-1996,Income_adj-1997,Income_adj-1998,Income_adj-1999,...,Income_adj-2008,Income_adj-2009,Income_adj-2010,Income_adj-2011,Income_adj-2012,Income_adj-2013,Income_adj-2014,Income_adj-2015,Income_adj-2016,Income_adj-2017
GeoName,,,,,,,,,,,,,,,,,,,,,
"Autauga, AL",25829.709197,26283.592893,26519.526215,26709.973702,27489.475358,27686.244094,28010.953983,28633.945520,29528.528147,30035.708812,...,33296.373279,33137.779348,33415.0,33274.675356,33279.102416,33195.559627,33782.947722,35503.778615,35895.513481,36014.111880
"Baldwin, AL",26411.970436,27143.329104,28142.123963,28831.681218,29617.066289,30266.001102,31050.396125,31837.534604,32966.589006,33209.693205,...,36208.134843,35267.143141,36282.0,36647.220020,36248.008641,35767.784922,36439.381488,38099.102900,39070.861367,39212.183518
"Barbour, AL",22506.315532,22702.159178,23224.595923,23162.225218,23438.812955,22767.106772,22387.916533,22647.934704,23612.923043,24887.964226,...,26006.335109,26515.915334,27834.0,27245.973068,26730.603239,28118.503586,27578.461645,28988.327858,28846.150321,29759.413218
"Bibb, AL",21141.588615,20646.477063,20972.542153,21029.954436,21608.437355,21965.851129,22086.334939,22247.146417,21610.286037,21563.460984,...,25302.448401,24179.205377,25091.0,25178.241603,25669.736875,25666.090824,26019.971377,26724.204120,26337.670918,26707.234138
"Blount, AL",23018.505218,22844.648047,22941.729266,22944.923737,23428.513414,23661.365302,24547.629879,24602.966380,25772.078773,25812.019064,...,28014.690998,27830.124109,27754.0,27499.956024,28112.483776,28257.036827,28884.572283,29428.088584,29310.414346,29985.368766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Sweetwater, WY",30215.854751,29520.811924,30286.937078,30548.966533,31747.599946,31729.723465,32554.134761,33899.896025,35271.561325,36043.374118,...,47780.234888,41667.431352,43231.0,45480.398224,45123.385698,44682.586125,45113.319343,45315.594814,43205.627669,44363.792102
"Teton, WY",59268.855394,57085.203642,59610.262466,64709.061149,64722.316653,69629.115381,71027.329458,73694.776374,83138.197742,90041.683457,...,154508.702842,127693.728578,145017.0,147094.907188,184513.817922,166767.812017,184225.155414,188520.271238,203845.523239,208039.230418
"Uinta, WY",25164.029441,25960.191189,25858.985944,26441.364927,26128.464561,26899.296588,26819.915156,28029.366579,28740.583460,29454.575174,...,41454.369591,39110.161995,38607.0,38620.919627,38028.782529,36590.560043,36098.576879,35816.579131,34402.781986,34513.375620


In [253]:
# #Income

# path_to_income = '/home/vlad/csv/US-Personal-Income/lapi1118_0.xlsx'
# income = (pd.read_excel(path_to_income
#                        #,sheet_name=1
#                        ,skiprows=3
# #                        ,index_col=0
#                        ,names = ['County',	'income-2015',	'income-2016'
#                                 ,'income-2017'
#                                 ,2017.1,2016.1,	2017.2,	2017.3]
#                       )
#          .iloc[:3216#,:4
#               ]
             
#          )

# df_list = [df.iloc[1:] for df in np.split(income,income[income.isnull().all(1)].index)][1:]

# df_list_res = []
# for i in range(len(df_list)):
#     #print(i)
#     df_list_res.append(df_list[i].assign(State = lambda x: x.iloc[0,0]).iloc[1:]
#                       )
# income_df = pd.concat(df_list_res,ignore_index=True)
# income_df['State-Code'] =income_df['State'].map(lambda x: state_codes.loc[x,'Postal Code'])
# # income_df['County'] = income_df['County'].map(str) +' County'

# income_df['County-State'] = income_df['County'].map(str) +', '+ income_df['State-Code'].map(str)
# income_df = income_df.set_index('County-State').iloc[:,:-2]
# income_df.sort_index()

In [7]:
#population data
path_to_pop = "/home/vlad/csv/US-Population/cbsa-est2018-alldata.xlsx"
pop = pd.read_excel(path_to_pop,sheet_name=1#,index_col=0
                     )
pop['NAME'] =pop['NAME'].str.strip()
pop = pop.set_index('NAME')
pop.index = [', '.join(ind.replace('County','').replace(' ','').strip().split(',')) for ind in pop.index]
pop.sort_index()

,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018
"Abbeville, SC",25417,25399,25310,25060,25000,24880,24775,24775,24633,24560,24541
"Aberdeen, SD",40602,40603,40717,40962,41576,42155,42220,42404,42849,43235,43191
"Aberdeen, WA",72797,72798,72843,72357,71770,71046,70783,71027,71572,72553,73901
"Abilene, TX",165252,165246,165583,166616,167447,167472,168355,169704,170018,170516,171451
"AcadiaParish, LA",61773,61787,61875,61853,61991,62294,62664,62681,62793,62514,62190
...,...,...,...,...,...,...,...,...,...,...,...
"Yuma, AZ",195751,195750,197138,202883,202589,202589,204283,205456,207520,209756,212128
"Yuma, AZ",195751,195750,197138,202883,202589,202589,204283,205456,207520,209756,212128
"Zanesville, OH",86074,86086,86213,86236,85786,85640,85900,86174,85973,86148,86183
"Zapata, TX",14018,14018,14087,14229,14285,14403,14404,14511,14456,14283,14190


In [18]:
# joint_full = unempdf.join(income_df).join(pop)

joint_full = unempdf.join(income_df_adj).join(pop)

In [19]:
joint_full.columns

Index(['Unemp-Pct-2014', 'Unemp-Pct-2015', 'Unemp-Pct-2016', 'Unemp-Pct-2017',
       'Unemp-Pct-2018', 'Unemp-Pct-2013', 'Unemp-Pct-2012', 'Unemp-Pct-2011',
       'Unemp-Pct-2010', 'Income_adj-1990', 'Income_adj-1991',
       'Income_adj-1992', 'Income_adj-1993', 'Income_adj-1994',
       'Income_adj-1995', 'Income_adj-1996', 'Income_adj-1997',
       'Income_adj-1998', 'Income_adj-1999', 'Income_adj-2000',
       'Income_adj-2001', 'Income_adj-2002', 'Income_adj-2003',
       'Income_adj-2004', 'Income_adj-2005', 'Income_adj-2006',
       'Income_adj-2007', 'Income_adj-2008', 'Income_adj-2009',
       'Income_adj-2010', 'Income_adj-2011', 'Income_adj-2012',
       'Income_adj-2013', 'Income_adj-2014', 'Income_adj-2015',
       'Income_adj-2016', 'Income_adj-2017', 'CENSUS2010POP',
       'ESTIMATESBASE2010', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POP

In [164]:
joint_full_nn = joint_full.loc[~joint_full.isnull().any(1)].drop(columns=['County'])

In [20]:
joint_full.loc['Cook, IL']

Unemp-Pct-2014       7.500000e+00
Unemp-Pct-2015       6.200000e+00
Unemp-Pct-2016       6.000000e+00
Unemp-Pct-2017       5.100000e+00
Unemp-Pct-2018       4.000000e+00
Unemp-Pct-2013       9.600000e+00
Unemp-Pct-2012       9.600000e+00
Unemp-Pct-2011       1.040000e+01
Unemp-Pct-2010       1.090000e+01
Income_adj-1990      3.719465e+04
Income_adj-1991      3.611693e+04
Income_adj-1992      3.717054e+04
Income_adj-1993      3.690654e+04
Income_adj-1994      3.743589e+04
Income_adj-1995      3.853324e+04
Income_adj-1996      3.930984e+04
Income_adj-1997      4.053532e+04
Income_adj-1998      4.240453e+04
Income_adj-1999      4.281541e+04
Income_adj-2000      4.466096e+04
Income_adj-2001      4.475258e+04
Income_adj-2002      4.462209e+04
Income_adj-2003      4.391695e+04
Income_adj-2004      4.434421e+04
Income_adj-2005      4.523127e+04
Income_adj-2006      4.695667e+04
Income_adj-2007      4.897957e+04
Income_adj-2008      4.755236e+04
Income_adj-2009      4.413932e+04
Income_adj-201

In [21]:
path_to_output ='/home/vlad/csv/SC-output/'

joint_full.reset_index().to_csv(path_to_output+'demography.csv',index=False)